In [ ]:
import scipy, GPy, popmachine
from GPy.kern import RBF
from derivative import RBFDerivative
import pandas as pd

In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
m = popmachine.machine.Machine()

for p in m.list(popmachine.models.Plate):
    print p

In [ ]:
ds = m.search(plates=['ecoli-eo499-propionicAcid-1'],include=['pH', 'propionicAcidmM', 'rep'], )
ds.data.shape

In [ ]:
ds.trim(3)
ds.data.shape

In [ ]:
ds.log()

In [ ]:
melt = ds.melt()
melt.head()

In [ ]:
plt.figure(figsize=(20,20))

g = melt.groupby(['pH','propionicAcidmM'])

for i, ph in enumerate(melt.pH.unique()):
    for j, pa in enumerate(melt.propionicAcidmM.unique()):
        temp = g.get_group((ph,pa))
        
        plt.subplot(melt.pH.unique().shape[0],melt.propionicAcidmM.unique().shape[0],i*melt.propionicAcidmM.unique().shape[0]+j+1)
        plt.title("pH=%.2lf,\nPAmM=%.2lf"%(ph, pa))
        
        g2 = temp.groupby('rep')
        for j,temp2 in g2:
            plt.plot(temp2.time, temp2.od)
        
        plt.ylim(np.nanmin(melt.od.values),np.nanmax(melt.od.values))
        
plt.tight_layout()
plt.savefig("figures/lund/ecoli.pdf",dpi=300,bbox_inches='tight')

In [ ]:
nrow,ncol = 3,4
plt.figure(figsize=(20,15))

g = melt.groupby(['pH','propionicAcidmM'])

for i, ph in enumerate(melt.pH.unique()):
    for j, pa in enumerate(melt.propionicAcidmM.unique()):
        temp = g.get_group((ph,pa))
        temp = temp.pivot('time', 'rep', 'od')
        
        plt.subplot(nrow,ncol,j+1)
        plt.title("PAmM=%.2lf"%(pa),fontsize=20)
        
        color = (ph-4)/3
        
        #plt.plot(temp.index, temp,c=plt.get_cmap()(color))
        plt.plot(temp.index, temp.median(1),c=plt.get_cmap()(color))
        plt.fill_between(temp.index, temp.min(1), temp.max(1),color=plt.get_cmap()(color),alpha=.4,label=ph)
        
        plt.ylim(melt.od.min(),melt.od.max())
        
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        
plt.tight_layout()

plt.subplot(nrow,ncol,1)
plt.legend(loc='best')

for c in range(ncol):
    plt.subplot(nrow,ncol,(nrow-1)*ncol+c+1)
    plt.xlabel("time (h)", fontsize=16)
    
for r in range(nrow):
    plt.subplot(nrow,ncol,r*ncol+1)
    plt.ylabel("log(OD)", fontsize=16)

plt.savefig("figures/lund/ecoli-shaded.pdf",dpi=300,bbox_inches='tight')

In [ ]:
melt.to_csv("data/normalized/lund/propionicAcid-ecoli/tidy.csv",index=False)